In [1]:
import os
import re
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.preprocessing import LabelEncoder

# Import functions for seed setting
from utils.seed import set_seed, init_weights

# Import the function for filtering Coq files
from data.data_functions import filter_coq_files

# Import the functions for preprocessing the data
from data.data_functions import (
    tokenize_coq_files_in_directory,
    create_sequences_and_labels,
    build_vocab,
    tokens_to_indices,
    split_data,
    CoqTokenDataset
)

# Import the function for training and evaluating the model
from train.eval import eval

# Import the models
from model.LSTM import LSTM
from model.transformer import Transformer
from model.n_gram import NGram
from model.LSTMFS import LSTMFS

In [2]:
# Set the seed
seed = 42
set_seed(seed)

## Extract the **Coq** files from **"math-comp"** folder

In [3]:
# Extract Coq files
input_folder = "data/math-comp"
output_folder = "data/coq_files"
if not os.path.isdir(output_folder):
    filter_coq_files(input_folder, output_folder)
elif not os.listdir(output_folder):
    filter_coq_files(input_folder, output_folder)

Copied: data/math-comp/mathcomp/ssreflect/ssrbool.v -> data/coq_files/ssrbool.v
Copied: data/math-comp/mathcomp/ssreflect/ssrfun.v -> data/coq_files/ssrfun.v
Copied: data/math-comp/mathcomp/ssreflect/seq.v -> data/coq_files/seq.v
Copied: data/math-comp/mathcomp/ssreflect/fintype.v -> data/coq_files/fintype.v
Copied: data/math-comp/mathcomp/ssreflect/bigop.v -> data/coq_files/bigop.v
Copied: data/math-comp/mathcomp/ssreflect/all_ssreflect.v -> data/coq_files/all_ssreflect.v
Copied: data/math-comp/mathcomp/ssreflect/generic_quotient.v -> data/coq_files/generic_quotient.v
Copied: data/math-comp/mathcomp/ssreflect/ssrnotations.v -> data/coq_files/ssrnotations.v
Copied: data/math-comp/mathcomp/ssreflect/fingraph.v -> data/coq_files/fingraph.v
Copied: data/math-comp/mathcomp/ssreflect/binomial.v -> data/coq_files/binomial.v
Copied: data/math-comp/mathcomp/ssreflect/finfun.v -> data/coq_files/finfun.v
Copied: data/math-comp/mathcomp/ssreflect/prime.v -> data/coq_files/prime.v
Copied: data/mat

## Preprocess the **dataset** and initialize the **dataloaders**

In [4]:
# Directory containing Coq files
directory = 'data/coq_files'

# Tokenize the Coq code
tokens, token_info = tokenize_coq_files_in_directory(directory)

# Create sequences of tokens and their labels
seq_length = 6
sequences, labels = create_sequences_and_labels(tokens, token_info, seq_length)

# Build the vocabulary and convert to indexed sequences
token_to_index = build_vocab(tokens)
indexed_sequences = tokens_to_indices(sequences, token_to_index)

# Split into train, validation, and test sets
train_seqs, train_labels, val_seqs, val_labels, test_seqs, test_labels = split_data(indexed_sequences, labels, seed)

# Create Dataset and DataLoader for each split
train_dataset = CoqTokenDataset(train_seqs, train_labels)
val_dataset = CoqTokenDataset(val_seqs, val_labels)
test_dataset = CoqTokenDataset(test_seqs, test_labels)

# Create the dataloaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, generator=torch.Generator().manual_seed(seed))
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("Data loaders initialized")

Data loaders initialized


## Load and evaluate the model

In [5]:
# Get the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
# Initialize the model architecture
pretrained_model_path = "pretrained_models/Transformer.pth"
model = torch.load(pretrained_model_path).to(device)

/tmp/ipykernel_10101/4154836927.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(pretrained_model_path).to(device)


In [7]:
# Calculate and print the overall accuracy
train_accuracy = eval(train_loader, model, device)
val_accuracy = eval(val_loader, model, device)
test_accuracy = eval(test_loader, model, device)
print(f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.3188, Validation Accuracy: 0.3178, Test Accuracy: 0.3179


### Final note:

**After training a model with these parameters:**

**Model Architecture: Transformer**

- **Hidden Dimension:** 768
- **Embedding Dimension:** 768
- **Number of Heads:** 8
- **Sequence Length:** 6
- **Batch Size:** 256
- **Learning Rate:** 0.00002
- **Epochs:** 15

---

**Final Results:**

- **Training Accuracy:** 99.72%
- **Validation Accuracy:** 98.65%
- **Test Accuracy:** 98.67%

---

After saving the trained model, I encountered a discrepancy: when reloading and testing the model, I was unable to reproduce these high accuracy scores (even after setting the random seed).